In [1]:
from typing import List
from Dataset.CustomDataset import AgeGroupAndAgeDataset, StandardDataset, AgeDatasetKL
from Dataset.CustomDataLoaders import CustomDataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from Utils import AAR, CSVUtils, AgeConversion
from Utils.Validator import Validator

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Caricamento del dataframe
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")

In [3]:
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
#########################
import pandas as pd
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
aug = CSVUtils.get_df_from_csv("./augumentation.csv", "./newAugmentationDataset/")
df_train = pd.concat([df_train, aug], ignore_index=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
#########################

from torchvision import transforms
import torch

transform_func = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(size=224),
    transforms.RandomHorizontalFlip(),
    transforms.RandAugment(2, 9),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

transform_func_val = transforms.Compose([
    transforms.Resize(224),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# Implementazione di un Dataset utilizzando "CustomDataset" per l'architettura con Film
cd_train = AgeDatasetKL(df_train, path_col="path", label_col="age", label_function="Linear", 
                        transform_func=transform_func)
cd_train.set_n_classes(81)
cd_train.set_starting_class(1)
dm_train = CustomDataLoader(cd_train)
dl_train = dm_train.get_unbalanced_dataloader(batch_size=64, shuffle=True, drop_last=True, num_workers=16, prefetch_factor=4, pin_memory=True)

cd_val = StandardDataset(df_val, path_col="path", label_col="age", label_function="CAE", transform_func=transform_func_val)
cd_val.set_n_classes(81)
cd_val.set_starting_class(1)
validator = Validator(cd_val, AgeConversion.EVAge, 32, num_workers=8, prefetch_factor=4)

In [4]:
from ResNetFilmed.resnet import ResNetFiLMed, BackBone, ResNetNotFiLMed, DoNothingLayer
from torchvision.models import resnet18, ResNet18_Weights, efficientnet_b0, EfficientNet_B0_Weights
import torch
from torch import optim
import torch.nn.functional as F
from torch import nn

####################################################
EPOCHS = 12
####################################################

backbone = resnet18(ResNet18_Weights.IMAGENET1K_V1)
backbone.fc = DoNothingLayer()
backbone.train()
backbone.requires_grad_(True)
backbone.to("cuda")
model_age = ResNetNotFiLMed(backbone, 81)
opt = optim.SGD(set([*backbone.parameters(), *model_age.fc0.parameters()]), lr=0.1, weight_decay=5e-4)
scheduler = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(dl_train), epochs=EPOCHS, three_phase=True)
kl = nn.KLDivLoss(reduction="batchmean")

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [5]:
best_val_aar = 20

for e in range(EPOCHS):
    with tqdm(dl_train, unit=" batch") as tepoch:
        for batch in tepoch:
            opt.zero_grad()
            x, y = batch
            x = x.to("cuda")
            y_age: torch.Tensor = y[0].to("cuda")
            y_age_kl: torch.Tensor = y[1].to("cuda")

            out_age = model_age(x)
            loss_age_kl: torch.Tensor = kl(F.log_softmax(out_age, dim=-1), y_age_kl)

            out = F.softmax(out_age, dim=-1)
            out = AgeConversion.EVAge(out).to("cuda")
            loss_age = torch.mean(torch.abs(y_age - out))

            loss = loss_age_kl + loss_age
            loss.backward()
            opt.step()
            scheduler.step()

            tepoch.set_postfix(loss_age_kl=loss_age_kl.detach().cpu().numpy(), loss_age=loss_age.detach().cpu().numpy())

    def forward_function(x):
        out = model_age(x)
        out = F.softmax(out, dim=-1)
        return out

    ae, mae, val_aar, val_aar_old = validator.validate_ext2(forward_function)
    print(ae, mae, val_aar, val_aar_old)

    if best_val_aar > mae:
        best_val_aar = mae
        torch.save(model_age.state_dict(), "./model_age_feature_simple.pt")
        print("Saved model")

100%|██████████| 4493/4493 [02:44<00:00, 27.24 batch/s]


{0: tensor(27.6070, dtype=torch.float64), 1: tensor(14.6324, dtype=torch.float64), 2: tensor(11.9121, dtype=torch.float64), 3: tensor(12.9548, dtype=torch.float64), 4: tensor(14.6832, dtype=torch.float64), 5: tensor(15.3561, dtype=torch.float64), 6: tensor(15.0982, dtype=torch.float64), 7: tensor(18.1049, dtype=torch.float64)} tensor(13.6782, dtype=torch.float64) tensor(0., dtype=torch.float64) tensor(0., dtype=torch.float64)
Saved model


100%|██████████| 4493/4493 [02:29<00:00, 30.07 batch/s]


{0: tensor(23.0902, dtype=torch.float64), 1: tensor(11.5271, dtype=torch.float64), 2: tensor(11.6141, dtype=torch.float64), 3: tensor(15.3388, dtype=torch.float64), 4: tensor(18.0509, dtype=torch.float64), 5: tensor(17.6549, dtype=torch.float64), 6: tensor(15.3298, dtype=torch.float64), 7: tensor(18.6179, dtype=torch.float64)} tensor(15.0515, dtype=torch.float64) tensor(1.1674, dtype=torch.float64) tensor(0., dtype=torch.float64)


 96%|█████████▋| 14155/14674 [19:13<00:39, 13.22 batch/s, loss_age=10.136554, loss_age_kl=1.9801809380046973] 

In [ ]:
####################################################
EPOCHS = 12
####################################################

dl_train = dm_train.get_balanced_class_dataloader(class_ranges=[(0, 11), (11, 21), (21, 31), (31, 41), (41, 51), (51, 61), (61, 71), (71, 91)], 
                                                  batch_size=64, num_workers=12, prefetch_factor=4, pin_memory=True)

model_age.load_state_dict(torch.load("./model_age_feature_simple.pt"))
opt = optim.SGD(set([*model_age.fc0.parameters()]), lr=0.1, weight_decay=5e-4)
scheduler = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(dl_train), epochs=EPOCHS, three_phase=True)

In [ ]:
def forward_function(x):
    out = model_age(x)
    out = F.softmax(out, dim=-1)
    return out

mae, val_aar, val_aar_old = validator.validate_ext(forward_function)
print(mae, val_aar, val_aar_old)

100%|██████████| 4493/4493 [02:21<00:00, 31.72 batch/s]


tensor(2.2409, dtype=torch.float64) tensor(4.9902, dtype=torch.float64) tensor(5.8763, dtype=torch.float64)


In [ ]:
best_val_aar = val_aar

for e in range(EPOCHS):
    with tqdm(dl_train, unit=" batch") as tepoch:
        for batch in tepoch:
            opt.zero_grad()
            x, y = batch
            x = x.to("cuda")
            y_age: torch.Tensor = y[0].to("cuda")
            y_age_kl: torch.Tensor = y[1].to("cuda")

            out_age = model_age(x)
            loss_age_kl: torch.Tensor = kl(F.log_softmax(out_age, dim=-1), y_age_kl)

            out = F.softmax(out_age, dim=-1)
            out = AgeConversion.EVAge(out).to("cuda")
            loss_age = torch.mean(torch.abs(y_age - out))

            loss = loss_age_kl + torch.square(loss_age - mae)
            loss.backward()
            opt.step()
            scheduler.step()

            tepoch.set_postfix(loss_age_kl=loss_age_kl.detach().cpu().numpy(), loss_age=loss_age.detach().cpu().numpy())

    def forward_function(x):
        out = model_age(x)
        out = F.softmax(out, dim=-1)
        return out

    val_aar, val_aar_old = validator.validate(forward_function)
    print(val_aar, val_aar_old)

    if val_aar > best_val_aar:
        best_val_aar = val_aar
        torch.save(model_age.state_dict(), "./model_age_classification_simple.pt")
        print("Saved model")

100%|██████████| 4493/4493 [01:58<00:00, 38.06 batch/s]


tensor(4.3330, dtype=torch.float64) tensor(3.5900, dtype=torch.float64)


100%|██████████| 4493/4493 [02:03<00:00, 36.30 batch/s]


tensor(3.4237, dtype=torch.float64) tensor(2.7052, dtype=torch.float64)


100%|██████████| 4493/4493 [02:19<00:00, 32.31 batch/s]


...

100%|██████████| 4493/4493 [01:53<00:00, 39.44 batch/s]


tensor(5.2628, dtype=torch.float64) tensor(4.7096, dtype=torch.float64)


100%|██████████| 4493/4493 [01:51<00:00, 40.46 batch/s]


tensor(5.3480, dtype=torch.float64) tensor(4.7429, dtype=torch.float64)


: 